In [ ]:
!pip install rdkit-pypi pandas scikit-learn tensorflow

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import roc_auc_score, accuracy_score
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, callbacks

# Load the dataset
file_path = '/content/tox21.csv'
data = pd.read_csv(file_path)

# Generate molecular descriptors using RDKit
descriptor_names = [desc[0] for desc in Descriptors.descList]
calculator = MolecularDescriptorCalculator(descriptor_names)

def generate_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return calculator.CalcDescriptors(mol)
    else:
        return [np.nan] * len(descriptor_names)

data['Descriptors'] = data['smiles'].apply(generate_descriptors)
descriptor_df = pd.DataFrame(data['Descriptors'].tolist(), columns=descriptor_names)

# Combine molecular descriptors with the original data
data = pd.concat([data, descriptor_df], axis=1).drop(columns=['Descriptors', 'smiles', 'mol_id'])

# Handle missing values
data.dropna(inplace=True)

# Prepare input (X) and target (Y) variables
X = data.drop(columns=['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER',
                       'NR-ER-LBD', 'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5',
                       'SR-HSE', 'SR-MMP', 'SR-p53'])
Y = data[['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD',
          'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53']]

# Feature selection
low_variance_filter = VarianceThreshold(threshold=0.1)
X = low_variance_filter.fit_transform(X)

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Build the improved multi-task neural network
input_layer = layers.Input(shape=(X_train.shape[1],))
hidden_layer = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01))(input_layer)
hidden_layer = layers.BatchNormalization()(hidden_layer)
hidden_layer = layers.Dropout(0.5)(hidden_layer)

hidden_layer = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(hidden_layer)
hidden_layer = layers.BatchNormalization()(hidden_layer)
hidden_layer = layers.Dropout(0.4)(hidden_layer)

hidden_layer = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(hidden_layer)
hidden_layer = layers.BatchNormalization()(hidden_layer)

# Output layers for multi-task predictions
outputs = [layers.Dense(1, activation='sigmoid', name=col)(hidden_layer) for col in Y.columns]

model = models.Model(inputs=input_layer, outputs=outputs)

# Compile the model
lr_schedule = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=['binary_crossentropy'] * len(Y.columns),
              metrics=['AUC'] * len(Y.columns))

# Train the model
history = model.fit(X_train, [Y_train[col] for col in Y.columns],
                    validation_split=0.2, epochs=100, batch_size=64,
                    callbacks=[lr_schedule, early_stopping])

# Evaluate the model
nn_preds = model.predict(X_test)
nn_roc_auc_scores = {
    col: roc_auc_score(Y_test[col], nn_preds[i]) for i, col in enumerate(Y.columns)
}
print("Neural Network ROC-AUC scores:", nn_roc_auc_scores)
# Calculate average AUC for Neural Network
average_auc_nn = np.mean(list(nn_roc_auc_scores.values()))
print(f"Average Neural Network ROC-AUC: {average_auc_nn:.4f}")

# Calculate accuracy scores for train and test data
nn_train_preds = model.predict(X_train)
nn_test_preds = model.predict(X_test)

train_accuracy_scores = {
    col: accuracy_score(Y_train[col], (nn_train_preds[i] > 0.5).astype(int)) for i, col in enumerate(Y.columns)
}
test_accuracy_scores = {
    col: accuracy_score(Y_test[col], (nn_test_preds[i] > 0.5).astype(int)) for i, col in enumerate(Y.columns)
}

# Print accuracy scores
print("Train Accuracy Scores:", train_accuracy_scores)
print("Test Accuracy Scores:", test_accuracy_scores)

# Calculate average accuracy scores
average_train_accuracy = np.mean(list(train_accuracy_scores.values()))
average_test_accuracy = np.mean(list(test_accuracy_scores.values()))

print(f"Average Train Accuracy: {average_train_accuracy:.4f}")
print(f"Average Test Accuracy: {average_test_accuracy:.4f}")

# Generate and display a sample output for a given SMILES string
def process_smiles(smiles):
    descriptors = generate_descriptors(smiles)
    descriptors = np.array(descriptors).reshape(1, -1)
    descriptors = low_variance_filter.transform(descriptors)
    descriptors = scaler.transform(descriptors)
    predictions = model.predict(descriptors)
    return {col: predictions[i][0] for i, col in enumerate(Y.columns)}

# Example SMILES string
sample_smiles = 'CCO'  # Ethanol
sample_output = process_smiles(sample_smiles)
print(f"Predicted targets for SMILES '{sample_smiles}': {sample_output}")


[12:15:41] WARNING: not removing hydrogen atom without neighbors
[12:15:41] WARNING: not removing hydrogen atom without neighbors
[12:15:41] WARNING: not removing hydrogen atom without neighbors


Epoch 1/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 15s 82ms/step - NR-AR-LBD_AUC: 0.4118 - NR-AR-LBD_loss: 0.8185 - NR-AR_AUC: 0.6206 - NR-AR_loss: 0.8448 - NR-AhR_AUC: 0.4969 - NR-AhR_loss: 0.8118 - NR-Aromatase_AUC: 0.5468 - NR-Aromatase_loss: 0.8240 - NR-ER-LBD_AUC: 0.4215 - NR-ER-LBD_loss: 0.8476 - NR-ER_AUC: 0.4902 - NR-ER_loss: 0.8129 - NR-PPAR-gamma_AUC: 0.5009 - NR-PPAR-gamma_loss: 0.8259 - SR-ARE_AUC: 0.5239 - SR-ARE_loss: 0.8265 - SR-ATAD5_AUC: 0.2678 - SR-ATAD5_loss: 0.8670 - SR-HSE_AUC: 0.5513 - SR-HSE_loss: 0.8103 - SR-MMP_AUC: 0.6003 - SR-MMP_loss: 0.8279 - SR-p53_AUC: 0.5646 - SR-p53_loss: 0.8306 - loss: 14.2388 - val_NR-AR-LBD_AUC: 0.7534 - val_NR-AR-LBD_loss: 0.6478 - val_NR-AR_AUC: 0.7153 - val_NR-AR_loss: 0.5981 - val_NR-AhR_AUC: 0.6069 - val_NR-AhR_loss: 0.6875 - val_NR-Aromatase_AUC: 0.6185 - val_NR-Aromatase_loss: 0.6605 - val_NR-ER-LBD_AUC: 0.6565 - val_NR-ER-LBD_loss: 0.6538 - val_NR-ER_AUC: 0.5386 - val_NR-ER_loss: 0.6322 - val_NR-PPAR-gamma_AUC: 0.3559 - val_NR-PPAR-gamma

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but VarianceThreshold was fitted with feature names
  warnings.warn(
